# Assignment 2 - Gesture Based UI

In [95]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from PIL import Image
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
print(tf.__version__)
print(tf.config.list_physical_devices('GPU'))

2.10.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Training CNN From Scratch - Default Image Colour
Image size = 128 x 128<br/>
Image colour = RGB<br/>



In [96]:
# Define constants
IMAGE_SIZE = (128, 128) #Image size of 128 x 128
BATCH_SIZE = 32
DATASET_PATH = "hagridset"
SEED = 399177 #G00399177

In [97]:
# This code sets up two TensorFlow Keras image datasets for training and validation/testing. The `train_dataset` is created from the 
# `DATASET_PATH` directory, with a 70/30 split for training and validation data. The `validation_and_test_dataset` is also created from
# the `DATASET_PATH` directory, with the same 70/30 split. The `test_dataset` is then created by taking the last 2/3 of the `validation_and_test_dataset`,
# and the `validation_dataset` is created by skipping the last 2/3 of the `validation_and_test_dataset`.
train_dataset = tf.keras.utils.image_dataset_from_directory(DATASET_PATH,
                                                            shuffle=True,
                                                            image_size=IMAGE_SIZE, validation_split=0.3
                                                           , subset='training', label_mode='categorical', 
                                                           seed=SEED, batch_size=BATCH_SIZE)

validation_and_test_dataset = tf.keras.utils.image_dataset_from_directory(DATASET_PATH,
                                                            shuffle=True,
                                                            image_size=IMAGE_SIZE, validation_split=0.3
                                                           , subset='validation',label_mode='categorical',
                                                           seed=SEED, batch_size=BATCH_SIZE)

val_batches = tf.data.experimental.cardinality(validation_and_test_dataset)

test_dataset_grayscale = validation_and_test_dataset.take((2*val_batches) // 3)
validation_dataset_grayscale = validation_and_test_dataset.skip((2*val_batches) // 3)


Found 125912 files belonging to 18 classes.
Using 88139 files for training.
Found 125912 files belonging to 18 classes.
Using 37773 files for validation.


In [98]:
# Get the number of classes from the data generator
NUM_CLASSES = len(train_dataset.class_names)
print("Classes: ",NUM_CLASSES)

Classes:  18


In [99]:

#The data augmentation layers is used to artificially expand the training dataset by
#applying random transformations to the input images, such as horizontal flipping and
#rotation. This can help the model generalize better and improve its performance on
#unseen data.

data_augmentation_layers = tf.keras.Sequential([
    # layers.RandomFlip("horizontal"),
    # layers.RandomRotation(0.1),
])

In [100]:
# CNN model from scratch
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1],3)))
model.add(data_augmentation_layers)
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [119]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 sequential_8 (Sequential)   multiple                  0         
                                                                 
 tf.math.truediv_1 (TFOpLamb  (None, 128, 128, 3)      0         
 da)                                                             
                                                                 
 tf.math.subtract_1 (TFOpLam  (None, 128, 128, 3)      0         
 bda)                                                            
                                                                 
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 global_average_pooling2d_1   (None, 512)              0   

In [101]:

# Configures an EarlyStopping callback for a Keras model.

# This callback monitors the validation loss during training and stops the training
# if the validation loss does not improve for 5 epochs. It also restores the
# weights of the model to the best performing weights during training.

# Args:
#     monitor (str): The metric to monitor for early stopping. In this case, it is
#         set to 'val_loss', which means the validation loss.
#     patience (int): The number of epochs to wait before stopping the training if
#         the monitored metric does not improve.
#     restore_best_weights (bool): If True, the model will be restored to the
#         weights that had the best value of the monitored metric.
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [102]:
# Trains the model on the provided training dataset and evaluates it on the validation dataset. The training runs for 10 epochs and uses the provided callback function to monitor and control the training process.

# Parameters:
# - train_dataset (tf.data.Dataset): The training dataset.
# - validation_dataset (tf.data.Dataset): The validation dataset.
# - epochs (int): The number of training epochs.
# - callbacks (list): A list of callback functions to use during training.
model.fit(train_dataset,  validation_data=(validation_dataset_grayscale), epochs=10, callbacks=[callback])

Epoch 1/10


2755/2755 [==============================] - 30s 11ms/step - loss: 2.1926 - accuracy: 0.2948 - val_loss: 1.7166 - val_accuracy: 0.4520
Epoch 2/10
2755/2755 [==============================] - 30s 11ms/step - loss: 1.4155 - accuracy: 0.5427 - val_loss: 1.3115 - val_accuracy: 0.5742
Epoch 3/10
2755/2755 [==============================] - 30s 11ms/step - loss: 1.0683 - accuracy: 0.6535 - val_loss: 1.2029 - val_accuracy: 0.6264
Epoch 4/10
2755/2755 [==============================] - 30s 11ms/step - loss: 0.8418 - accuracy: 0.7270 - val_loss: 1.2123 - val_accuracy: 0.6366
Epoch 5/10
2755/2755 [==============================] - 30s 11ms/step - loss: 0.6682 - accuracy: 0.7811 - val_loss: 1.3201 - val_accuracy: 0.6431
Epoch 6/10
2755/2755 [==============================] - 30s 11ms/step - loss: 0.5225 - accuracy: 0.8285 - val_loss: 1.5034 - val_accuracy: 0.6386
Epoch 7/10
2755/2755 [==============================] - 29s 11ms/step - loss: 0.4169 - accuracy: 0.8616 - val_loss: 1.7840 - val_accura

64x64: With the contrast at 0.5 epoch tens accuracy was 0.5199 </br>
64x64: with just flipping and rotation it had an accuracy of 0.5697</br>



In [103]:
# Evaluates the model on the test dataset and returns a dictionary of evaluation metrics.

# Arguments:
# - test_dataset: The dataset to evaluate the model on.
# - return_dict: If True, the method returns a dictionary of evaluation metrics. If False, it returns a list of evaluation metrics.
model.evaluate(test_dataset_grayscale, return_dict=True)

787/787 [==============================] - 5s 6ms/step - loss: 1.1936 - accuracy: 0.6260


{'loss': 1.1936277151107788, 'accuracy': 0.625952959060669}

In [104]:
class_names = train_dataset.class_names


In [105]:
my_image = np.array(Image.open("MyHandStop.jpg").resize((IMAGE_SIZE[0],IMAGE_SIZE[1])))
my_image = my_image.reshape(1,IMAGE_SIZE[0],IMAGE_SIZE[1],3)

prediction_of_my_image = model.predict(my_image)
pd.DataFrame(prediction_of_my_image, columns=class_names).idxmax(axis=1)

1/1 [==============================] - 0s 37ms/step


0    stop_inverted
dtype: object

# CNN from Scatch - Grayscale Images

In [120]:
# Define constants
IMAGE_SIZE = (128, 128) #Image size of 128 x 128
BATCH_SIZE = 32
DATASET_PATH = "hagridset"
SEED = 399177 #G00399177

In [121]:
# Define a function to preprocess images by converting them to grayscale
def preprocess_image(image, label):
    # Convert image to grayscale
    image = tf.image.rgb_to_grayscale(image)
    # Normalize the pixel values to the range [0,1]
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


In [122]:
# This code sets up two TensorFlow Keras image datasets for training and validation/testing. The `train_dataset` is created from the 
# `DATASET_PATH` directory, with a 70/30 split for training and validation data. The `validation_and_test_dataset` is also created from
# the `DATASET_PATH` directory, with the same 70/30 split. The `test_dataset` is then created by taking the last 2/3 of the `validation_and_test_dataset`,
# and the `validation_dataset` is created by skipping the last 2/3 of the `validation_and_test_dataset`.
train_dataset = tf.keras.utils.image_dataset_from_directory(DATASET_PATH,
                                                            shuffle=True,
                                                            image_size=IMAGE_SIZE, validation_split=0.3
                                                           , subset='training', label_mode='categorical', 
                                                           seed=SEED, batch_size=BATCH_SIZE)
NUM_CLASSES = len(train_dataset.class_names)
CLASS_NAMES = train_dataset.class_names
# Apply preprocessing to train_dataset
train_dataset_grayscale = train_dataset.map(preprocess_image)

validation_and_test_dataset = tf.keras.utils.image_dataset_from_directory(DATASET_PATH,
                                                            shuffle=True,
                                                            image_size=IMAGE_SIZE, validation_split=0.3
                                                           , subset='validation',label_mode='categorical',
                                                           seed=SEED, batch_size=BATCH_SIZE)
# Apply preprocessing to validation_and_test_dataset
validation_and_test_dataset_grayscale = validation_and_test_dataset.map(preprocess_image)

val_batches = tf.data.experimental.cardinality(validation_and_test_dataset)

test_dataset_grayscale = validation_and_test_dataset.take((2*val_batches) // 3)
validation_dataset_grayscale = validation_and_test_dataset.skip((2*val_batches) // 3)


Found 125912 files belonging to 18 classes.
Using 88139 files for training.
Found 125912 files belonging to 18 classes.
Using 37773 files for validation.


In [123]:
# CNN model from scratch
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1],1)))
model.add(data_augmentation_layers)
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [124]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [125]:
model.fit(train_dataset_grayscale,  validation_data=(validation_dataset_grayscale), epochs=10, callbacks=[callback])

Epoch 1/10
2755/2755 [==============================] - 21s 8ms/step - loss: 2.8904 - accuracy: 0.0583 - val_loss: 2.8905 - val_accuracy: 0.0549
Epoch 2/10
2755/2755 [==============================] - 21s 8ms/step - loss: 2.8904 - accuracy: 0.0585 - val_loss: 2.8906 - val_accuracy: 0.0550
Epoch 3/10
2755/2755 [==============================] - 21s 8ms/step - loss: 2.8904 - accuracy: 0.0582 - val_loss: 2.8906 - val_accuracy: 0.0550
Epoch 4/10
2755/2755 [==============================] - 21s 8ms/step - loss: 2.8904 - accuracy: 0.0582 - val_loss: 2.8906 - val_accuracy: 0.0548
Epoch 5/10
2755/2755 [==============================] - 21s 8ms/step - loss: 2.8904 - accuracy: 0.0583 - val_loss: 2.8906 - val_accuracy: 0.0548
Epoch 6/10
2755/2755 [==============================] - 21s 8ms/step - loss: 2.8904 - accuracy: 0.0584 - val_loss: 2.8906 - val_accuracy: 0.0548


In [ ]:
model.summary()

In [112]:
model.evaluate(test_dataset_grayscale, return_dict=True)

787/787 [==============================] - 4s 4ms/step - loss: 2.8904 - accuracy: 0.0581


{'loss': 2.8904130458831787, 'accuracy': 0.05805273354053497}

In [113]:
# Load and preprocess the image
my_image = Image.open("MyHandStop.jpg").resize((IMAGE_SIZE[0], IMAGE_SIZE[1]))

# Convert the image to grayscale if necessary
my_image = my_image.convert("L")

# Convert the image to numpy array
my_image_array = np.array(my_image)

# Expand dimensions to match the model's input shape
my_image_array = np.expand_dims(my_image_array, axis=0)
my_image_array = np.expand_dims(my_image_array, axis=-1)

# Predict the class probabilities
prediction_of_my_image = model.predict(my_image_array)

# Get the index of the predicted class (the class with the highest probability)
predicted_class_index = np.argmax(prediction_of_my_image)

# Map the predicted class index to the corresponding class name
predicted_class_name = CLASS_NAMES[predicted_class_index]

print("Predicted class:", predicted_class_name)

1/1 [==============================] - 0s 37ms/step
Predicted class: stop_inverted


# Transfer Learning

In [114]:
inputs = tf.keras.Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1],3))

In [115]:
base_model = tf.keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1],3),
    include_top=False)
base_model.trainable = False
x = data_augmentation_layers(inputs)
x = tf.keras.applications.xception.preprocess_input(x)
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(NUM_CLASSES)(x)
model = tf.keras.Model(inputs, outputs)

In [116]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [117]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [118]:
model.fit(train_dataset,  validation_data=(validation_dataset_grayscale), epochs=10, callbacks=[callback])

Epoch 1/10


ValueError: in user code:

    File "c:\Users\Conor\anaconda3\envs\py310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Conor\anaconda3\envs\py310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\Conor\anaconda3\envs\py310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\Conor\anaconda3\envs\py310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\Conor\anaconda3\envs\py310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Conor\anaconda3\envs\py310\lib\site-packages\keras\engine\input_spec.py", line 277, in assert_input_

In [ ]:
model.evaluate(test_dataset_grayscale, return_dict=True)

epoch ten transfer learning: val_accuracy: 0.3462